# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_window': [48], # [6, 12, 24, 48, 72, 96],
                            'vol_q_high': np.arange(0.1, 1, 0.05), 
                            'vol_q_low': np.arange(0.1, 1, 0.05),
                            'window_low_bound': [1], 
                            'window_high_bound': [7],
                            'first_candle': [0.5], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0], # np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': [16], # np.arange(2, 50, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 324


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 324/324 [1:02:43<00:00, 11.62s/it]


# Check local statistics

In [2]:
# result
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,48,0.75,0.2,1,7,0.5,0,16,0,269,2.155646,1.066667,310.868729,286.933333,1.6767,1.754,1.8001,1.8638,1.9433,2.0275,2.1034,2.109,2.1322,2.1332,2.1348,2.1758,2.198,2.2183,2.3062,2.2977,2.2709,2.3172,2.3271,2.3219,2.2803,2.2897,2.4403,2.6141,0.11,0.21,0.31,0.41,0.5,0.59,0.69,0.78,0.88,0.98,1.07,1.18,1.27,1.36,1.46,1.49,1.42,1.49,1.53,1.55,1.61,1.53,1.57,1.61


In [2]:
# candles new
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
234,Pattern_Trend,48,0.75,0.10,1,7,0.5,0,16,0,58,2.072854,1.436250,62.225542,83.302500,1.1446,1.4550,1.7086,1.6540,1.5030,1.5913,1.7006,1.8069,1.8695,1.9441,1.9656,1.9528,2.0326,2.3265,2.3490,2.3625,2.4196,2.4690,2.5192,2.6029,2.6224,2.5803,2.5908,2.5777,0.19,0.34,0.47,0.62,0.72,0.84,0.99,1.12,1.24,1.36,1.50,1.62,1.71,1.81,1.93,2.03,2.14,2.19,2.24,2.12,2.14,1.66,1.73,1.76
252,Pattern_Trend,48,0.80,0.10,1,7,0.5,0,16,0,45,2.382588,1.291250,62.216438,58.106250,2.0796,2.0765,2.0834,1.9516,1.7458,1.8830,1.9723,2.0503,2.1670,2.2581,2.2557,2.1976,2.2696,2.6218,2.6303,2.6559,2.7143,2.7243,2.8166,2.7992,2.8111,2.8035,2.8169,2.7977,0.20,0.35,0.47,0.61,0.70,0.81,0.93,1.04,1.15,1.27,1.40,1.50,1.59,1.68,1.79,1.88,1.98,2.01,2.05,1.86,1.85,1.21,1.31,1.35
235,Pattern_Trend,48,0.75,0.15,1,7,0.5,0,16,0,67,2.052054,1.267083,70.487629,84.894583,1.1174,1.4108,1.6252,1.5779,1.4392,1.5262,1.6237,1.7205,1.7944,1.8773,1.8971,1.8753,1.9492,2.2105,2.3148,2.3501,2.4873,2.5331,2.5683,2.6536,2.6708,2.6690,2.6842,2.6734,0.16,0.29,0.39,0.52,0.61,0.71,0.83,0.94,1.05,1.15,1.27,1.38,1.46,1.56,1.67,1.77,1.89,1.95,2.01,1.92,1.96,1.57,1.65,1.70
180,Pattern_Trend,48,0.60,0.10,1,7,0.5,0,16,0,87,1.864163,1.223333,75.182138,106.430000,0.9847,1.2522,1.4743,1.4567,1.3771,1.4468,1.6206,1.6880,1.7068,1.7616,1.7882,1.8067,1.8452,2.1097,2.1586,2.1197,2.1464,2.2395,2.2710,2.2274,2.2695,2.2723,2.3110,2.4059,0.16,0.30,0.41,0.54,0.64,0.73,0.84,0.94,1.03,1.14,1.26,1.35,1.50,1.56,1.63,1.70,1.76,1.87,1.89,1.81,1.82,1.48,1.52,1.48
239,Pattern_Trend,48,0.75,0.35,1,7,0.5,0,16,0,121,3.473058,1.210000,299.240058,146.410000,0.9909,1.1695,1.3215,1.3015,4.6480,4.4226,4.1054,3.9650,3.9523,3.8369,3.7559,3.7457,3.7137,3.8545,3.8978,3.8907,3.9276,3.9270,3.8829,3.8928,3.8510,3.8034,3.7744,3.7224,0.14,0.26,0.36,0.46,0.58,0.69,0.79,0.87,0.97,1.07,1.19,1.29,1.37,1.48,1.59,1.67,1.77,1.82,1.87,1.84,1.87,1.67,1.71,1.71
236,Pattern_Trend,48,0.75,0.20,1,7,0.5,0,16,0,80,1.818633,1.202500,65.490667,96.200000,1.0045,1.2522,1.4745,1.4151,1.3092,1.4058,1.4887,1.5747,1.6450,1.6850,1.6714,1.6766,1.7220,1.9460,2.0420,2.0776,2.1950,2.2372,2.2727,2.3173,2.2933,2.3083,2.3183,2.3148,0.15,0.28,0.38,0.50,0.59,0.69,0.81,0.91,1.02,1.11,1.22,1.33,1.40,1.49,1.59,1.68,1.78,1.82,1.87,1.79,1.82,1.49,1.56,1.58
162,Pattern_Trend,48,0.55,0.10,1,7,0.5,0,16,0,102,1.956687,1.189583,97.582125,121.337500,0.9657,1.2542,1.4494,1.4118,1.3711,1.5020,1.6762,1.7719,1.7799,1.8341,1.8677,1.9159,1.9817,2.2654,2.3047,2.2779,2.3126,2.4227,2.4509,2.4026,2.4397,2.4159,2.4302,2.4563,0.15,0.29,0.40,0.51,0.61,0.70,0.80,0.90,0.99,1.09,1.22,1.31,1.45,1.53,1.60,1.65,1.71,1.80,1.81,1.74,1.76,1.49,1.53,1.51
216,Pattern_Trend,48,0.70,0.10,1,7,0.5,0,16,0,71,2.054150,1.185417,74.844650,84.164583,1.0312,1.3418,1.5623,1.5640,1.4722,1.5455,1.7314,1.8242,1.8354,1.9014,1.9406,1.9528,2.0262,2.3173,2.3930,2.3450,2.3787,2.4943,2.5371,2.5545,2.6064,2.5958,2.6301,2.7184,0.15,0.28,0.37,0.48,0.57,0.65,0.75,0.85,0.92,1.01,1.13,1.23,1.40,1.47,1.57,1.65,1.74,1.90,1.95,1.86,1.89,1.49,1.56,1.58
240,Pattern_Trend,48,0.75,0.40,1,7,0.5,0,16,0,131,3.471013,1.150000,

In [3]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
8,Pattern_Trend,48,0.8,0.2,1,6,0.90,0,34,0,82,2.203733,1.342083,98.706133,110.050833,1.8215,1.6317,1.4874,1.5300,1.6772,1.9023,1.9994,2.1298,2.0879,2.1761,2.2460,2.3532,2.3860,2.4969,2.5576,2.4853,2.4560,2.5181,2.5211,2.5333,2.4379,2.5013,2.4753,2.4783,0.17,0.30,0.43,0.55,0.65,0.77,0.91,1.04,1.18,1.32,1.46,1.62,1.76,1.94,2.09,2.04,1.70,1.81,1.85,1.91,1.96,1.57,1.61,1.57
7,Pattern_Trend,48,0.8,0.2,1,6,0.85,0,34,0,100,2.250979,1.312500,125.097917,131.250000,1.8121,1.6337,1.5433,1.6878,1.8152,1.9630,2.0482,2.1665,2.1415,2.2158,2.2753,2.3987,2.4276,2.5296,2.5818,2.5299,2.5147,2.5739,2.5611,2.5601,2.4799,2.5344,2.4977,2.5317,0.15,0.27,0.39,0.51,0.61,0.72,0.84,0.96,1.10,1.23,1.37,1.54,1.68,1.85,1.99,1.96,1.69,1.80,1.84,1.90,1.98,1.67,1.72,1.73
9,Pattern_Trend,48,0.8,0.2,1,6,0.95,0,34,0,69,2.139504,1.275000,78.625788,87.975000,1.8499,1.6223,1.4729,1.5191,1.7118,1.9164,2.0285,2.1392,2.0556,2.1780,2.2816,2.3996,2.3867,2.4942,2.5063,2.3865,2.3459,2.3432,2.3432,2.3574,2.2532,2.2780,2.2467,2.2319,0.16,0.28,0.41,0.54,0.66,0.79,0.94,1.08,1.22,1.36,1.50,1.66,1.80,1.99,2.14,2.04,1.56,1.65,1.67,1.71,1.75,1.24,1.26,1.19
6,Pattern_Trend,48,0.8,0.2,1,6,0.80,0,34,0,113,2.092946,1.157917,123.502879,130.844583,1.6581,1.5370,1.4607,1.5779,1.6672,1.7741,1.8748,1.9734,1.9647,2.0277,2.0868,2.1614,2.1960,2.2797,2.3282,2.2395,2.2264,2.2778,2.2669,2.2759,2.2321,2.2987,2.6791,3.1666,0.14,0.25,0.36,0.47,0.55,0.65,0.75,0.85,0.97,1.09,1.21,1.36,1.48,1.62,1.74,1.71,1.47,1.56,1.60,1.65,1.72,1.45,1.54,1.60
5,Pattern_Trend,48,0.8,0.2,1,6,0.75,0,34,0,126,2.012488,1.062917,127.573425,133.927500,1.5752,1.4568,1.4063,1.5723,1.6499,1.7777,1.8703,1.9250,1.9379,1.9930,2.0610,2.1467,2.1654,2.1788,2.1814,2.1148,2.1004,2.1451,2.1470,2.1572,2.1238,2.1854,2.5152,2.9131,0.13,0.23,0.33,0.43,0.51,0.60,0.69,0.79,0.90,1.02,1.13,1.27,1.38,1.51,1.59,1.56,1.34,1.42,1.45,1.50,1.56,1.32,1.40,1.45
4,Pattern_Trend,48,0.8,0.2,1,6,0.70,0,34,0,145,2.044462,1.010417,151.447062,146.510417,1.6402,1.4754,1.4745,1.6127,1.6684,1.7116,1.8151,1.9336,1.9738,2.0086,2.0636,2.1692,2.1802,2.2027,2.2519,2.2145,2.1617,2.2040,2.2039,2.2249,2.1919,2.2539,2.5445,2.8863,0.12,0.21,0.31,0.41,0.48,0.57,0.66,0.75,0.85,0.96,1.06,1.18,1.28,1.39,1.48,1.46,1.28,1.35,1.39,1.44,1.50,1.31,1.38,1.43
1,Pattern_Trend,48,0.8,0.2,1,6,0.55,0,34,0,198,2.170887,0.958333,231.835725,189.750000,1.7866,1.7401,1.7750,1.8959,1.9766,2.0136,2.0962,2.1336,2.1568,2.1619,2.1588,2.2256,2.2261,2.2375,2.3297,2.2762,2.2562,2.2793,2.3182,2.3006,2.2573,2.2906,2.4872,2.7217,0.11,0.20,0.30,0.39,0.46,0.55,0.63,0.72,0.81,0.90,0.99,1.10,1.19,1.28,1.36,1.36,1.25,1.32,1.35,1.35,1.41,1.27,1.33,1.37
3,Pattern_Trend,48,0.8,0.2,1,6,0.65,0,34,0,171,2.023467,0.910833,175.012800,155.752500,1.6157,1.5302,1.5816,1.6965,1.7312,1.8088,1.8889,1.9822,1.9933,1.9799,2.0457,2.1330,2.1351,2.1592,2.2146,2.1715,2.1228,2.1462,2.1423,2.1290,2.0925,2.1570,2.4027,2.7033,0.10,0.19,0.28,0.37,0.43,0.50,0.59,0.67,0.76,0.85,0.94,1.05,1.14,1.24,1.31,1.30,1.15,1.22,1.26,1.33,1.39,1.21,1.27,1.31
0,Pattern_Trend,48,0.8,0.2,1,6,0.50,0,34,0,222,2.143925,0.889167,253.951350,197

In [2]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,2,0,167,2.013317,1.176667,169.223883,196.503333,1.6929,1.7829,1.8600,1.9275,1.9580,2.0704,1.9786,2.0028,1.9556,1.9416,1.9617,2.0293,2.0369,1.9973,2.0839,2.0910,2.0782,2.0829,2.0953,2.0482,2.0162,2.0143,2.2057,2.4084,0.11,0.22,0.34,0.45,0.53,0.64,0.74,0.84,0.93,1.02,1.12,1.23,1.33,1.41,1.50,1.56,1.64,1.72,1.75,1.76,1.81,1.83,1.86,1.90
2,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,6,0,184,2.091933,1.078750,200.915733,198.490000,1.7879,1.8450,1.8671,1.9732,2.0391,2.1375,2.1129,2.1134,2.0573,2.0173,2.0177,2.0782,2.0839,2.0594,2.1553,2.1512,2.1075,2.1428,2.1553,2.1483,2.1238,2.1501,2.3515,2.5307,0.11,0.21,0.31,0.41,0.50,0.60,0.70,0.79,0.88,0.96,1.05,1.14,1.23,1.30,1.39,1.44,1.53,1.59,1.64,1.64,1.70,1.54,1.59,1.64
1,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,4,0,177,1.999375,1.025833,176.889375,181.572500,1.6413,1.7446,1.7988,1.9087,1.9427,2.0481,2.0207,2.0240,1.9751,1.9440,1.9529,2.0118,2.0219,1.9782,2.0690,2.0522,2.0083,2.0434,2.0560,2.0280,2.0111,2.0388,2.2375,2.4279,0.10,0.19,0.29,0.39,0.46,0.55,0.65,0.73,0.81,0.90,0.98,1.06,1.15,1.21,1.30,1.35,1.42,1.49,1.54,1.54,1.59,1.60,1.64,1.68
7,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,16,0,211,2.156696,0.995417,244.062821,210.032917,1.6870,1.7091,1.7424,1.8455,1.9181,2.0364,2.1082,2.1438,2.1563,2.1693,2.1818,2.2205,2.2357,2.2343,2.3416,2.2853,2.2497,2.2825,2.3120,2.2911,2.2442,2.2651,2.4462,2.6546,0.11,0.20,0.30,0.39,0.47,0.56,0.65,0.74,0.83,0.94,1.03,1.14,1.23,1.31,1.40,1.41,1.31,1.37,1.42,1.42,1.48,1.35,1.40,1.43
4,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,10,0,193,2.058425,0.981250,204.276025,189.381250,1.7036,1.7323,1.7447,1.8467,1.9348,2.0422,2.0093,2.0462,2.0481,2.0417,2.0511,2.0915,2.0815,2.0800,2.1878,2.1514,2.1057,2.1368,2.1543,2.1224,2.1032,2.1305,2.3275,2.5289,0.10,0.19,0.28,0.37,0.44,0.52,0.61,0.69,0.78,0.87,0.95,1.04,1.12,1.19,1.28,1.30,1.40,1.46,1.51,1.51,1.57,1.42,1.46,1.49
6,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,14,0,208,2.040275,0.977500,216.377200,203.320000,1.5787,1.6331,1.6680,1.7693,1.8483,1.9626,1.9747,2.0157,2.0308,2.0470,2.0636,2.0999,2.1003,2.1005,2.2040,2.1546,2.1294,2.1613,2.1732,2.1423,2.1236,2.1504,2.3248,2.5105,0.11,0.20,0.30,0.39,0.47,0.55,0.64,0.73,0.82,0.91,1.00,1.11,1.19,1.27,1.36,1.41,1.30,1.36,1.41,1.41,1.46,1.32,1.36,1.38
8,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,18,0,212,2.152654,0.967083,244.362683,205.021667,1.6779,1.6906,1.7178,1.8264,1.8916,2.0109,2.0838,2.1328,2.1454,2.1588,2.1715,2.2354,2.2499,2.2483,2.3577,2.3010,2.2630,2.2943,2.3234,2.3032,2.2456,2.2566,2.4357,2.6421,0.11,0.20,0.29,0.38,0.46,0.54,0.63,0.72,0.81,0.90,0.99,1.10,1.19,1.27,1.37,1.37,1.28,1.34,1.39,1.39,1.44,1.31,1.35,1.38
3,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,8,0,192,2.066858,0.963750,204.836800,185.040000,1.6741,1.7163,1.7618,1.8843,1.9611,2.0554,2.0230,2.0398,1.9980,1.9903,1.9976,2.0654,2.0822,2.0820,2.1921,2.1703,2.1268,2.1652,2.1823,2.1798,2.1458,2.1759,2.3781,2.5570,0.10,0.18,0.27,0.36,0.43,0.51,0.60,0.68,0.76,0.85,0.93,1.02,1.10,1.17,1.26,1.29,1.37,1.43,1.48,1.48,1.54,1.39,1.44,1.49
5,Pattern_Trend,48,0.8,0.2,1,6,0.5,0,12,0,203,2.056421,0.938750,214.453429,190.566250,

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
252,Pattern_Trend,48,0.80,0.10,1,6,0.5,0,16,0,129,2.276092,1.249167,164.615825,161.142500,1.6189,1.6483,1.7396,1.9109,1.9990,2.0585,2.1308,2.1301,2.2336,2.3070,2.2814,2.2747,2.2731,2.3108,2.4542,2.4447,2.4584,2.4872,2.5616,2.5524,2.4579,2.4696,2.7465,3.0770,0.12,0.23,0.36,0.47,0.56,0.68,0.79,0.90,1.01,1.14,1.23,1.34,1.43,1.53,1.64,1.67,1.79,1.88,1.92,1.89,1.97,1.75,1.82,1.86
234,Pattern_Trend,48,0.75,0.10,1,6,0.5,0,16,0,150,2.135579,1.222917,170.336875,183.437500,1.6006,1.5007,1.6494,1.7890,1.8838,1.9122,2.0074,2.0190,2.0979,2.1560,2.1228,2.1246,2.1220,2.1616,2.2855,2.2782,2.2902,2.3281,2.3674,2.4068,2.3345,2.3516,2.5890,2.8756,0.12,0.23,0.35,0.46,0.57,0.68,0.78,0.89,1.00,1.12,1.21,1.31,1.40,1.49,1.60,1.64,1.72,1.81,1.85,1.84,1.92,1.73,1.79,1.84
216,Pattern_Trend,48,0.70,0.10,1,6,0.5,0,16,0,195,2.177333,1.073750,229.580000,209.381250,1.2934,1.3445,1.4846,1.6159,1.7220,1.7650,1.8333,1.9223,1.9947,2.0550,2.0588,2.0607,2.0652,2.1006,2.2032,2.1799,2.1810,2.7826,2.8662,2.8774,2.8262,2.8443,2.9805,3.1987,0.10,0.18,0.28,0.37,0.46,0.55,0.63,0.73,0.82,0.92,1.01,1.10,1.19,1.28,1.38,1.42,1.49,1.58,1.64,1.66,1.75,1.67,1.75,1.81
253,Pattern_Trend,48,0.80,0.15,1,6,0.5,0,16,0,177,2.143650,1.040833,202.426050,184.227500,1.5663,1.6308,1.7315,1.8549,1.9522,2.0722,2.1559,2.1733,2.1994,2.2202,2.2384,2.2350,2.2152,2.2026,2.2986,2.2307,2.2058,2.2333,2.2815,2.2552,2.1949,2.2196,2.4248,2.6553,0.12,0.22,0.32,0.43,0.52,0.62,0.72,0.82,0.93,1.04,1.13,1.23,1.32,1.40,1.50,1.50,1.35,1.41,1.43,1.43,1.48,1.31,1.36,1.39
235,Pattern_Trend,48,0.75,0.15,1,6,0.5,0,16,0,198,2.051983,1.040000,208.292700,205.920000,1.5581,1.5171,1.6616,1.7666,1.8676,1.9540,2.0543,2.0789,2.0970,2.1149,2.1212,2.1231,2.1053,2.1020,2.1893,2.1322,2.1114,2.1452,2.1721,2.1841,2.1344,2.1606,2.3438,2.5528,0.12,0.21,0.32,0.42,0.52,0.62,0.73,0.82,0.93,1.03,1.12,1.22,1.30,1.39,1.48,1.49,1.34,1.40,1.43,1.43,1.49,1.34,1.39,1.42
236,Pattern_Trend,48,0.75,0.20,1,6,0.5,0,16,0,230,2.103429,0.999167,253.788708,229.808333,1.6660,1.5962,1.6858,1.7723,1.8730,1.9700,2.0594,2.0949,2.1131,2.1263,2.1284,2.1649,2.1764,2.1807,2.2795,2.2283,2.1980,2.2380,2.2491,2.2583,2.2158,2.2317,2.3947,2.5815,0.11,0.20,0.30,0.39,0.48,0.57,0.66,0.75,0.84,0.94,1.03,1.14,1.23,1.31,1.41,1.42,1.31,1.38,1.41,1.42,1.48,1.36,1.40,1.44
198,Pattern_Trend,48,0.65,0.10,1,6,0.5,0,16,0,219,2.088862,0.995833,238.460887,218.087500,1.3788,1.3467,1.4717,1.5705,1.6614,1.7114,1.7938,1.8656,1.9404,1.9845,1.9805,1.9873,1.9945,2.0382,2.1368,2.0881,2.0779,2.6080,2.6863,2.6898,2.6494,2.6690,2.8128,2.9893,0.09,0.17,0.26,0.34,0.42,0.50,0.58,0.67,0.75,0.84,0.93,1.02,1.10,1.18,1.28,1.32,1.39,1.47,1.52,1.54,1.62,1.56,1.65,1.70
254,Pattern_Trend,48,0.80,0.20,1,6,0.5,0,16,0,209,2.189137,0.994583,248.529737,207.867917,1.6844,1.7053,1.7485,1.8480,1.9463,2.0736,2.1475,2.1784,2.2038,2.2191,2.2308,2.2673,2.2813,2.2785,2.3863,2.3261,2.2904,2.3259,2.3539,2.3292,2.2779,2.2912,2.4712,2.6744,0.11,0.20,0.30,0.39,0.47,0.56,0.66,0.74,0.84,0.94,1.03,1.14,1.23,1.31,1.41,1.42,1.32,1.38,1.41,1.41,1.47,1.34,1.38,1.41
237,Pattern_Trend,48,0.75,0.25,1,6,0.5,0,16,0,251,

In [2]:
# vol window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
5,Pattern_Trend,96,0.75,0.2,1,6,0.5,0,16,0,195,3.798392,1.068333,545.686375,208.325000,1.4229,1.4346,1.5547,1.7147,4.5705,4.2906,4.2727,4.2334,4.1923,4.0756,4.0289,3.9559,3.9415,3.8836,3.9956,3.9513,3.8999,4.5464,4.5763,4.4627,4.3330,4.3753,4.6093,4.8397,0.09,0.17,0.25,0.34,0.42,0.51,0.59,0.68,0.78,0.88,0.98,1.08,1.17,1.25,1.36,1.43,1.52,1.61,1.67,1.69,1.79,1.70,1.79,1.89
3,Pattern_Trend,48,0.75,0.2,1,6,0.5,0,16,0,235,2.109792,0.994167,260.801042,233.629167,1.6758,1.6006,1.6874,1.7731,1.8732,1.9726,2.0614,2.0967,2.1232,2.1333,2.1352,2.1713,2.1824,2.1865,2.2848,2.2336,2.2033,2.2431,2.2568,2.2663,2.2236,2.2464,2.4088,2.5956,0.11,0.20,0.29,0.39,0.47,0.56,0.66,0.75,0.84,0.94,1.02,1.13,1.22,1.30,1.40,1.41,1.30,1.37,1.40,1.41,1.47,1.37,1.41,1.44
2,Pattern_Trend,24,0.75,0.2,1,6,0.5,0,16,0,223,2.156483,0.970417,257.895783,216.402917,1.8166,1.7141,1.7544,1.7548,1.8907,2.0541,2.1001,2.2539,2.2400,2.2525,2.2649,2.2389,2.2324,2.2608,2.3058,2.2781,2.1980,2.1699,2.2456,2.2694,2.2205,2.2236,2.4021,2.6144,0.10,0.18,0.27,0.35,0.43,0.52,0.62,0.71,0.81,0.91,0.99,1.10,1.19,1.28,1.36,1.36,1.23,1.29,1.32,1.38,1.43,1.45,1.49,1.52
1,Pattern_Trend,12,0.75,0.2,1,6,0.5,0,16,0,209,2.391371,0.924167,290.796504,193.150833,1.6280,1.5720,1.9572,2.0492,2.1235,2.3085,2.3206,2.3067,2.2760,2.3076,2.3122,2.3003,2.2999,2.2526,2.2955,2.2610,2.2720,2.8609,2.9109,2.8749,2.8097,2.8113,3.0266,3.2558,0.09,0.16,0.25,0.33,0.41,0.50,0.58,0.67,0.76,0.85,0.92,1.00,1.08,1.16,1.24,1.28,1.13,1.20,1.26,1.33,1.40,1.45,1.53,1.60
4,Pattern_Trend,72,0.75,0.2,1,6,0.5,0,16,0,213,2.140171,0.918750,242.856387,195.693750,1.3944,1.3969,1.5381,1.6726,1.7782,1.7861,1.8585,1.9007,1.9574,1.9567,1.9690,1.9821,2.0183,2.0463,2.1611,2.1593,2.1359,2.7314,2.7965,2.7465,2.6714,2.6919,2.8975,3.1173,0.08,0.15,0.23,0.31,0.38,0.45,0.53,0.61,0.69,0.78,0.85,0.93,1.01,1.08,1.16,1.23,1.30,1.38,1.43,1.44,1.52,1.43,1.50,1.58
0,Pattern_Trend,6,0.75,0.2,1,6,0.5,0,16,0,225,1.786033,0.735417,176.857500,165.468750,1.4202,1.2244,1.4673,1.4650,1.5240,1.6564,1.6488,1.6941,1.7562,1.7760,1.7898,1.8084,1.8078,1.8052,1.8395,1.8648,1.9566,1.9605,1.9977,1.9913,1.9769,1.9630,2.1439,2.3270,0.08,0.15,0.22,0.29,0.37,0.44,0.51,0.58,0.66,0.75,0.82,0.90,0.94,0.99,1.03,1.08,0.93,0.97,0.99,1.02,1.05,0.92,0.96,1.00


In [3]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
11,Pattern_Trend,48,0.75,0.2,3,7,0.5,0,16,0,214,2.248671,1.134583,267.215558,242.800833,1.7036,1.8611,1.9111,2.0019,2.1010,2.1493,2.1603,2.1363,2.1627,2.1643,2.2028,2.2875,2.3105,2.3479,2.4461,2.4543,2.4022,2.4180,2.4005,2.3808,2.3382,2.3386,2.5309,2.7582,0.12,0.22,0.33,0.43,0.53,0.63,0.74,0.83,0.94,1.05,1.15,1.28,1.38,1.48,1.58,1.61,1.50,1.58,1.61,1.62,1.69,1.59,1.64,1.70
8,Pattern_Trend,48,0.75,0.2,3,4,0.5,0,16,0,95,2.176233,1.108333,111.742167,105.291667,1.5976,1.7106,1.7768,1.9168,2.1574,2.2332,2.2335,2.2172,2.2329,2.3165,2.3563,2.3658,2.3795,2.4072,2.3989,2.3733,2.2615,2.2329,2.1914,2.1835,2.1583,2.1682,2.1776,2.1827,0.12,0.24,0.36,0.47,0.59,0.70,0.81,0.91,1.03,1.17,1.29,1.44,1.55,1.67,1.78,1.73,1.37,1.39,1.43,1.47,1.52,1.16,1.19,1.21
3,Pattern_Trend,48,0.75,0.2,1,7,0.5,0,16,0,269,2.155646,1.066667,310.868729,286.933333,1.6767,1.7540,1.8001,1.8638,1.9433,2.0275,2.1034,2.1090,2.1322,2.1332,2.1348,2.1758,2.1980,2.2183,2.3062,2.2977,2.2709,2.3172,2.3271,2.3219,2.2803,2.2897,2.4403,2.6141,0.11,0.21,0.31,0.41,0.50,0.59,0.69,0.78,0.88,0.98,1.07,1.18,1.27,1.36,1.46,1.49,1.42,1.49,1.53,1.55,1.61,1.53,1.57,1.61
9,Pattern_Trend,48,0.75,0.2,3,5,0.5,0,16,0,134,2.104596,1.057917,148.015842,141.760833,1.6377,1.5701,1.7324,1.8899,2.0560,2.0893,2.1096,2.1043,2.1062,2.1675,2.2179,2.2892,2.3094,2.3215,2.3695,2.2924,2.2159,2.2046,2.1836,2.1921,2.1334,2.1112,2.1024,2.1042,0.12,0.22,0.33,0.42,0.52,0.62,0.72,0.81,0.92,1.04,1.15,1.28,1.39,1.49,1.60,1.58,1.35,1.40,1.42,1.47,1.53,1.31,1.34,1.36
7,Pattern_Trend,48,0.75,0.2,2,7,0.5,0,16,0,260,2.189721,1.055000,309.327417,274.300000,1.7355,1.7867,1.8275,1.8944,1.9709,2.0484,2.1088,2.1015,2.1394,2.1379,2.1405,2.1805,2.2021,2.2518,2.3411,2.3440,2.3196,2.3677,2.3770,2.3863,2.3413,2.3497,2.5070,2.6937,0.11,0.21,0.30,0.40,0.49,0.59,0.68,0.77,0.87,0.96,1.05,1.16,1.25,1.34,1.44,1.47,1.40,1.48,1.52,1.54,1.60,1.52,1.56,1.61
10,Pattern_Trend,48,0.75,0.2,3,6,0.5,0,16,0,178,2.225588,1.054583,218.154575,187.715833,1.7301,1.6907,1.8072,1.9304,2.0601,2.1216,2.1406,2.1511,2.1822,2.1948,2.2401,2.3219,2.3288,2.3466,2.4641,2.4156,2.3527,2.3549,2.3380,2.3353,2.2909,2.3078,2.5257,2.7829,0.12,0.21,0.32,0.41,0.50,0.60,0.70,0.79,0.90,1.00,1.11,1.23,1.32,1.42,1.52,1.52,1.37,1.44,1.46,1.47,1.54,1.39,1.45,1.52
0,Pattern_Trend,48,0.75,0.2,1,4,0.5,0,16,0,152,2.012858,0.999167,153.954467,151.873333,1.5494,1.5651,1.6081,1.6848,1.8343,1.9564,2.0658,2.0984,2.1142,2.1645,2.1427,2.1116,2.1275,2.1322,2.1479,2.1091,2.0662,2.1043,2.1175,2.1310,2.0995,2.1208,2.1292,2.1281,0.11,0.21,0.31,0.42,0.52,0.62,0.71,0.80,0.90,1.01,1.10,1.23,1.32,1.40,1.51,1.47,1.26,1.29,1.35,1.38,1.43,1.20,1.22,1.21
2,Pattern_Trend,48,0.75,0.2,1,6,0.5,0,16,0,235,2.109792,0.994167,260.801042,233.629167,1.6758,1.6006,1.6874,1.7731,1.8732,1.9726,2.0614,2.0967,2.1232,2.1333,2.1352,2.1713,2.1824,2.1865,2.2848,2.2336,2.2033,2.2431,2.2568,2.2663,2.2236,2.2464,2.4088,2.5956,0.11,0.20,0.29,0.39,0.47,0.56,0.66,0.75,0.84,0.94,1.02,1.13,1.22,1.30,1.40,1.41,1.30,1.37,1.40,1.41,1.47,1.37,1.41,1.44
1,Pattern_Trend,48,0.75,0.2,1,5,0.5,0,16,0,192,1.995863,0.982083,191.2056

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'vol_window': [48],
                            'vol_q_high': [0.75],
                            'vol_q_low': [0.2],
                            'window_low_bound': [1], 
                            'window_high_bound': [7],
                            'first_candle': [0.5],
                            'second_candle': [0],
                            },
                'Trend': {
                            'timeperiod': [16], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)